In [ ]:
#installing packages
!pip install -q flwr[simulation] torch torchvision matplotlib

In [ ]:
#importing libraries
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
import torchvision.models as models

from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.datasets import ImageFolder
from PIL import Image

import flwr as fl
from flwr.common import Metrics

device = "cuda" if torch.cuda.is_available() else "cpu"
device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
#print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

In [ ]:
CLASSES = (
    "MildDemented",
    "ModerateDemented",
    "NonDemented",
    "VeryMildDemented",
)

In [ ]:
NUM_CLIENTS = 3
BATCH_SIZE = 64
NBR_CLASSES = 4
epochs = 5
num_rounds=3

In [ ]:
# Defining Custom dataset class (still gotta know if its necessary or not)
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = os.listdir(self.root_dir)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image


In [ ]:

!wget -O AugmentedAlzheimerDataset.zip https://dl.dropboxusercontent.com/s/g933yjln90vhrzk/AugmentedAlzheimerDataset.zip


In [ ]:
!unzip AugmentedAlzheimerDataset.zip -d destination_folder


In [ ]:
def load_datasets():
    import os

    # Download and transform (train and test)
    transform = transforms.Compose(
            [transforms.ToTensor(),transforms.Resize((224,224)), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )

    root_dir = "/content/destination_folder/AugmentedAlzheimerDataset"
    print("Listing root directory contents:")
    print(os.listdir(root_dir))

    for class_name in ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']:
        class_dir = os.path.join(root_dir, class_name)
        print(f"Listing contents of {class_name} directory:")
        print(os.listdir(class_dir))

    # Load Dataset
    dataset = datasets.ImageFolder(root=root_dir, transform=transform)

    # Define Training and Testing Sets
    trainset_size=int(0.9*len(dataset))
    testset_size= len(dataset)-trainset_size
    train_dataset, test_dataset = random_split(dataset,[trainset_size,testset_size])

    # Split training set into 3 partitions to simulate the individual dataset
    partition_size = len(train_dataset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    clientdatasets = random_split(train_dataset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in clientdatasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets()


In [ ]:
images, labels = next(iter(trainloaders[0]))

# Reshape and convert images to a NumPy array
# matplotlib requires images with the shape (height, width, 3)
images = images.permute(0, 2, 3, 1).numpy()
# Denormalize
images = images / 2 + 0.5

# Create a figure and a grid of subplots
fig, axs = plt.subplots(4, 8, figsize=(12, 6))

# Loop over the images and plot them
for i, ax in enumerate(axs.flat):
    ax.imshow(images[i])
    ax.set_title(CLASSES[labels[i]])
    ax.axis("off")

# Show the plot
fig.tight_layout()
plt.show()

In [ ]:
#define model architecture
model = models.densenet121(pretrained=True)
#model = models.densenet121(progress=True)
model.classifier = nn.Linear(1024, NBR_CLASSES)

In [ ]:

def train(model, trainloader, epochs: int, parameters: List[np.ndarray] = None):
    """Train the network on the training set."""
    model_parameters = [val.cpu().numpy() for _, val in model.state_dict().items()]

    if parameters is not None:
        for param in parameters:
            if param.size == 0:
                print("Empty numpy array found!")
        params_dict = zip(model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        model.load_state_dict(state_dict, strict=True)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    model.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(model(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")

def test(model, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    model.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
trainloader = trainloaders[0]
valloader = valloaders[0]
model.to(device)

for epoch in range(epochs):
    train(model, trainloader, 1)
    loss, accuracy = test(model, valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

loss, accuracy = test(model, testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

In [ ]:

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, trainloader, valloader):
        self.cid = cid
        self.model = model
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)
        train(self.model, self.trainloader, epochs=1)
        return self.get_parameters(config), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)
        loss, accuracy = test(self.model, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
    model_copy = models.densenet121(pretrained=True)
    model_copy.classifier = nn.Linear(1024, NBR_CLASSES)
    model_copy.to(device)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, model_copy, trainloader, valloader)


In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [ ]:
# The `evaluate` function will be by Flower called after every round
"""def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    model,
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    model = model.to(device)
    valloader = valloaders[0]
    set_parameters(model, parameters)  # Update model with the latest parameters
    loss, accuracy = test(model, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}"""


def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    model,
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    print (f"-------/{server_round}/--------{config}")
    model = model.to(device)
    valloader = valloaders[0]

    # Set model parameters directly without calling set_parameters function
    for param in parameters:
        if param.size == 0:
            print("Empty numpy array found!")

    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

    loss, accuracy = test(model, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    torch.cuda.empty_cache()
    return loss, {"accuracy": accuracy}


In [ ]:

def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

In [ ]:
# Create an instance of the model and get the parameters

model_parameters = [val.cpu().numpy() for _, val in model.state_dict().items()]
# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=2,
    min_evaluate_clients=2,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(model_parameters),
    on_fit_config_fn=fit_config  # Pass the fit_config function
    # evaluate_fn=evaluate(server_round=0, parameters=model_parameters, config=fit_config(0), model=model),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if device.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=2),  # six rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-24 18:20:02,055 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)
2023-04-24 18:20:03,727	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-04-24 18:20:05,049 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'object_store_memory': 26672431104.0, 'memory': 53344862208.0, 'accelerator_type:A100': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 12.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'object_store_memory': 26672431104.0, 'memory': 53344862208.0, 'accelerator_type:A100': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 12.0}
INFO flwr 2023-04-24 18:20:05,053 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-24 18:20:05,055 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provide

(launch_and_fit pid=21402) [Client 1] fit, config: {'server_round': 1, 'local_epochs': 1}


(launch_and_fit pid=21402) /usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
(launch_and_fit pid=21402)   warnings.warn(


(launch_and_fit pid=21402) Epoch 1: train loss 0.01770535483956337, accuracy 0.8913469921534438
(launch_and_fit pid=21402) [Client 1] get_parameters


(pid=21900) 2023-04-24 18:21:37.319534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(launch_and_fit pid=21900) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_fit pid=21900)   warnings.warn(
(launch_and_fit pid=21900) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
(launch_and_fit pid=21900)   warnings.warn(msg)


...

(launch_and_fit pid=24149) [Client 2] get_parameters


DEBUG flwr 2023-04-24 18:29:41,414 | server.py:168 | evaluate_round 2: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 2: strategy sampled 3 clients (out of 3)
(pid=24628) 2023-04-24 18:29:43.214778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(launch_and_evaluate pid=24628) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(launch_and_evaluate pid=24628)   warnings.warn(
(launch_and_evaluate pid=24628) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-

(launch_and_evaluate pid=24628) [Client 0] evaluate, config: {}


ERROR flwr 2023-04-24 18:29:47,117 | ray_client_proxy.py:104 | ray::launch_and_evaluate() (pid=24628, ip=172.28.0.12)
  File "/usr/local/lib/python3.9/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 160, in launch_and_evaluate
    return maybe_call_evaluate(
  File "/usr/local/lib/python3.9/dist-packages/flwr/client/client.py", line 205, in maybe_call_evaluate
    return client.evaluate(evaluate_ins)
  File "/usr/local/lib/python3.9/dist-packages/flwr/client/app.py", line 321, in _evaluate
    results = self.numpy_client.evaluate(parameters, ins.config)  # type: ignore
  File "<ipython-input-11-fd86fd150091>", line 25, in evaluate
  File "<ipython-input-9-336b4cc2d635>", line 41, in test
  File "/usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/torchvision/models/densenet.py", line 213, in forward
    features = self.features(x)
  Fil

History (loss, distributed):
	round 1: 0.00725825301910445